In [1]:
import sys.process._

In [2]:
val url = "https://raw.githubusercontent.com/rupeshtr78/machine_learning/master/data/fakefriends.csv"

s"wget $url".!

--2020-02-20 14:30:44--  https://raw.githubusercontent.com/rupeshtr78/machine_learning/master/data/fakefriends.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8754 (8.5K) [text/plain]
Saving to: 'fakefriends.csv'

     0K ........                                              100% 60.4M=0s

2020-02-20 14:30:44 (60.4 MB/s) - 'fakefriends.csv' saved [8754/8754]



url = https://raw.githubusercontent.com/rupeshtr78/machine_learning/master/data/fakefriends.csv


0

In [3]:
"ls"!

fakefriends.csv
logs
spark-events
user-libs


0

## Define Class

In [4]:
case class Person(ID:Int, name:String, age:Int, numFriends:Int)

defined class Person


### Split by seperator and map to class

In [5]:
  def mapper(line:String): Person = {
    val fields = line.split(',')  
    
    val person:Person = Person(fields(0).toInt, fields(1), fields(2).toInt, fields(3).toInt)
    return person
  }

mapper: (line: String)Person


In [6]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.sql._
import org.apache.log4j._

In [7]:
    val spark = SparkSession
      .builder
      .appName("SparkSQL")
      .getOrCreate()

spark = org.apache.spark.sql.SparkSession@2121c092


org.apache.spark.sql.SparkSession@2121c092

In [8]:
val lines = spark.sparkContext.textFile("fakefriends.csv")

lines = fakefriends.csv MapPartitionsRDD[1] at textFile at <console>:42


fakefriends.csv MapPartitionsRDD[1] at textFile at <console>:42

In [9]:
val people = lines.map(mapper)

people = MapPartitionsRDD[2] at map at <console>:48


MapPartitionsRDD[2] at map at <console>:48

In [10]:
val schemaPeople = people.toDS

schemaPeople = [ID: int, name: string ... 2 more fields]


[ID: int, name: string ... 2 more fields]

In [11]:
schemaPeople.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)
 |-- numFriends: integer (nullable = false)



In [12]:
schemaPeople.createOrReplaceTempView("people")

In [13]:
val teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

teenagers = [ID: int, name: string ... 2 more fields]


[ID: int, name: string ... 2 more fields]

In [14]:
val results = teenagers.collect()

results = Array([21,Miles,19,268], [52,Beverly,19,269], [54,Brunt,19,5], [106,Beverly,18,499], [115,Dukat,18,397], [133,Quark,19,265], [136,Will,19,335], [225,Elim,19,106], [304,Will,19,404], [341,Data,18,326], [366,Keiko,19,119], [373,Quark,19,272], [377,Beverly,18,418], [404,Kasidy,18,24], [409,Nog,19,267], [439,Data,18,417], [444,Keiko,18,472], [492,Dukat,19,36], [494,Kasidy,18,194])


Array([21,Miles,19,268], [52,Beverly,19,269], [54,Brunt,19,5], [106,Beverly,18,499], [115,Dukat,18,397], [133,Quark,19,265], [136,Will,19,335], [225,Elim,19,106], [304,Will,19,404], [341,Data,18,326], [366,Keiko,19,119], [373,Quark,19,272], [377,Beverly,18,418], [404,Kasidy,18,24], [409,Nog,19,267], [439,Data,18,417], [444,Keiko,18,472], [492,Dukat,19,36], [494,Kasidy,18,194])

In [15]:
results.foreach(println)

[21,Miles,19,268]
[52,Beverly,19,269]
[54,Brunt,19,5]
[106,Beverly,18,499]
[115,Dukat,18,397]
[133,Quark,19,265]
[136,Will,19,335]
[225,Elim,19,106]
[304,Will,19,404]
[341,Data,18,326]
[366,Keiko,19,119]
[373,Quark,19,272]
[377,Beverly,18,418]
[404,Kasidy,18,24]
[409,Nog,19,267]
[439,Data,18,417]
[444,Keiko,18,472]
[492,Dukat,19,36]
[494,Kasidy,18,194]


## Adding Cache

In [16]:
// val people = lines.map(mapper)
// val schemaPeople = people.toDS
val people_cache = lines.map(mapper).toDS().cache()

people_cache = [ID: int, name: string ... 2 more fields]


[ID: int, name: string ... 2 more fields]

In [17]:
people_cache.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)
 |-- numFriends: integer (nullable = false)



In [18]:
people_cache.select("name").show()

+--------+
|    name|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
|  Deanna|
|   Quark|
|  Weyoun|
|  Gowron|
|    Will|
|  Jadzia|
|    Hugh|
|     Odo|
|     Ben|
|   Keiko|
|Jean-Luc|
|    Hugh|
|     Rom|
|  Weyoun|
|     Odo|
|Jean-Luc|
|  Geordi|
+--------+
only showing top 20 rows



In [19]:
people_cache.filter(people_cache("age") < 21).show()

+---+-------+---+----------+
| ID|   name|age|numFriends|
+---+-------+---+----------+
| 21|  Miles| 19|       268|
| 48|    Nog| 20|         1|
| 52|Beverly| 19|       269|
| 54|  Brunt| 19|         5|
| 60| Geordi| 20|       100|
| 73|  Brunt| 20|       384|
|106|Beverly| 18|       499|
|115|  Dukat| 18|       397|
|133|  Quark| 19|       265|
|136|   Will| 19|       335|
|225|   Elim| 19|       106|
|304|   Will| 19|       404|
|327| Julian| 20|        63|
|341|   Data| 18|       326|
|349| Kasidy| 20|       277|
|366|  Keiko| 19|       119|
|373|  Quark| 19|       272|
|377|Beverly| 18|       418|
|404| Kasidy| 18|        24|
|409|    Nog| 19|       267|
+---+-------+---+----------+
only showing top 20 rows



In [20]:
people_cache.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
| 26|   17|
| 27|    8|
| 44|   12|
| 22|    7|
| 47|    9|
| 52|   11|
| 40|   17|
| 20|    5|
| 57|   12|
| 54|   13|
| 48|   10|
| 19|   11|
| 64|   12|
| 41|    9|
| 43|    7|
+---+-----+
only showing top 20 rows



In [21]:
people_cache.select(people_cache("name"), people_cache("age") + 10).show()

+--------+----------+
|    name|(age + 10)|
+--------+----------+
|    Will|        43|
|Jean-Luc|        36|
|    Hugh|        65|
|  Deanna|        50|
|   Quark|        78|
|  Weyoun|        69|
|  Gowron|        47|
|    Will|        64|
|  Jadzia|        48|
|    Hugh|        37|
|     Odo|        63|
|     Ben|        67|
|   Keiko|        64|
|Jean-Luc|        66|
|    Hugh|        53|
|     Rom|        46|
|  Weyoun|        32|
|     Odo|        45|
|Jean-Luc|        55|
|  Geordi|        70|
+--------+----------+
only showing top 20 rows

